# 01 - OpenAI Interview Question Generation
`Author: Abdlazeez Jimoh`

In [1]:
import json
from pprint import pprint

from openai import OpenAI
from typing import Literal, TypedDict

In [2]:
OPENAI_API_KEY = "YOUR_API_KEY"

In [3]:
class Question(TypedDict):
    question: str
    type: Literal["personal", "role-specific", "behavioral", "situational"]

In [4]:
class OpenAIQuestionGeneratorAgent():
    def __init__(self):
        super().__init__()
        self.client = OpenAI(api_key=OPENAI_API_KEY)
        self.system_prompt = """You are a non-technical interviewer that interviews \
across the following categories:
- personal
- role-specific
- behavioural
- situational

You will be provided with a candidate's description.

Generate {n_questions} questions, ensuring that there is a question for each category \
and the questions should be based on the candidate's description.

* You answer strictly as a list of JSON objects. Don't include any other verbose texts, \
and don't include the markdown syntax anywhere.

JSON format:
[
    {{"question": "<personal_question>", "type": "personal"}},
    {{"question": "<role_specific_question>", "type": "role-specific"}},
    {{"question": "<behavioural_question>", "type": "behavioural"}},
    {{"question": "<situational_question>", "type": "situational"}},
    ...more questions to make up {n_questions} questions
]"""

        self.user_prompt = "Candidate Description:\n{description}"

    def __call__(self, description: str, n_questions: int = 4) -> list[Question] | None:
        """
        Generate interview questions based on the given description.

        Args:
            description (str): The description used as input for question generation.
            n_questions (int, optional): The number of questions to generate. Defaults to 4.

        Returns:
            list[Question] | None: A list of generated interview questions or None if an error occurs.
        """

        # Generate questions
        questions = self._generate(description, n_questions)

        return questions

    def run(self, description: str, n_questions: int = 4) -> list[Question] | None:
        """
        Generate interview questions based on the given description.

        Args:
            description (str): The description used as input for question generation.
            n_questions (int, optional): The number of questions to generate. Defaults to 4.

        Returns:
            list[Question] | None: A list of generated interview questions or None if an error occurs.
        """

        # Generate questions
        questions = self._generate(description, n_questions)

        return questions

    def _generate(self, description: str, n_questions: int) -> list[Question] | None:
        """
        Generate interview questions based on the given description.

        Args:
            description (str): The description used as input for question generation.
            n_questions (int): The number of questions to generate.

        Returns:
            list[Question] | None: A list of generated interview questions or None if an error occurs.
        """

        try:
            # Ensure that there are at least 4 questions
            if n_questions < 4:
                n_questions = 4

            output = self.client.chat.completions.create(
                model="gpt-3.5-turbo-1106",
                messages=[
                    {
                        "role": "system",
                        "content": self.system_prompt.format(n_questions=n_questions),
                    },
                    {
                        "role": "user",
                        "content": self.user_prompt.format(description=description),
                    },
                ],
                temperature=0.5,
                max_tokens=1024,
                top_p=1,
                frequency_penalty=0,
                presence_penalty=0,
            )
            questions = json.loads(output.choices[0].message.content or "[]")

            return questions
        except Exception as e:
            return None

In [5]:
question_generator = OpenAIQuestionGeneratorAgent()
questions = question_generator.run(
    "a software engineer at a startup in San Francisco. I have 5 years of experience and I'm looking for a new job.",
    5
)

pprint(questions)

[{'question': 'What motivated you to pursue a career in software engineering?',
  'type': 'personal'},
 {'question': 'Can you describe a challenging technical problem you solved at '
              'your current startup and how you approached it?',
  'type': 'role-specific'},
 {'question': 'Tell me about a time when you had to work on a project with a '
              'tight deadline. How did you manage your time and prioritize '
              'tasks?',
  'type': 'behavioural'},
 {'question': 'You are joining a new team where the existing codebase is '
              'outdated. How would you approach the situation and contribute '
              'effectively to the project?',
  'type': 'situational'}]


In [6]:
questions = question_generator.run(
    "a data scientist from India. I have 3 years of experience. I've worked on a variety of projects, including a recommendation engine for a large e-commerce company. I've led a team of 5 data scientists and engineers.",
    8
)

pprint(questions)

[{'question': 'What inspired you to pursue a career in data science?',
  'type': 'personal'},
 {'question': 'Can you discuss a specific project where you implemented a '
              'recommendation engine? What were the challenges and how did you '
              'overcome them?',
  'type': 'role-specific'},
 {'question': 'Tell me about a time when you had to lead a team to achieve a '
              'challenging goal. How did you motivate and guide your team '
              'members?',
  'type': 'behavioural'},
 {'question': 'You have experience working on diverse projects. How do you '
              'prioritize and manage your time when juggling multiple '
              'projects?',
  'type': 'situational'},
 {'question': 'What are your career goals and how do you see yourself '
              'progressing in the field of data science?',
  'type': 'personal'},
 {'question': 'What techniques or methodologies do you typically use when '
              'approaching a new data science proj